In [1]:
from pyspark.context import  SparkContext
from pyspark.sql.functions import col, lower
from pyspark.sql import SQLContext
import re
sc = SparkContext('local','test1')
sql = SQLContext(sc)

In [2]:
from pyspark.sql.functions import lit
dems_df =  sql.read.text("dems.txt")
gop_df = sql.read.text("gop.txt")
neu_df=sql.read.text("NonPolitical.txt")

In [3]:
corpus_df = dems_df.select("value", lit(1).alias("label")).union(gop_df.select("value", lit(0).alias("label"))).union(neu_df.select("value", lit(2).alias("label")))

In [4]:
corpus_df.select("*").limit(20).show()

+--------------------+-----+
|               value|label|
+--------------------+-----+
|This week @senate...|    1|
|Health care profe...|    1|
|RT @SeemaNanda: G...|    1|
|Republicans keep ...|    1|
|RT @SpeakerPelosi...|    1|
|While the preside...|    1|
|You are not alone...|    1|
|RT @DNCWarRoom: W...|    1|
|RT @DNCWarRoom: T...|    1|
|RT @DNCWarRoom: T...|    1|
|LISTEN. TO. HEALT...|    1|
|RT @SeemaNanda: B...|    1|
|This is a HUGE wi...|    1|
|RT @SenSherrodBro...|    1|
|RT @WisDems: Make...|    1|
|Trump had warning...|    1|
|RT @DemConvention...|    1|
|Abortion is healt...|    1|
|RT @RepLucyMcBath...|    1|
|Get counted. Get ...|    1|
+--------------------+-----+



In [5]:
from pyspark.sql.functions import udf,lower,col,trim
from pyspark.sql.types import FloatType,StringType,IntegerType
def clean_text(text):
    text=re.sub(r'@[A-Za-z0-9]+','',text).strip() #remove mentions
    text=re.sub(r'#\S+','',text).strip() #removing #symbol
    text=re.sub(r'RT[\s]+','',text).strip()
    text=re.sub(r'[?|$|.|!|;|:|&|"|,|""|*|-|(|)_]','',text).strip()
    #text=re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)+',"",text).strip()
    text=re.sub(r'http[a-zA-Z]*',' ',text).strip
    return text.strip()

In [6]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string).strip()

In [7]:
#value
clean_udf_str=udf(lambda z: clean_text(z), StringType())
corpus_df=corpus_df.select("label",clean_udf_str("value").alias("value"))
emoji_udf_str=udf(lambda z: remove_emoji(z), StringType())
corpus_df=corpus_df.select("label",emoji_udf_str('value').alias('value'))
corpus_df=corpus_df.select(trim(lower(col('value'))).alias("value"),"label")

In [8]:
#After Preprocessing
corpus_df.select("*").limit(20).show()

Py4JJavaError: An error occurred while calling o107.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 4, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Users\pavan\anaconda3\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 377, in main
  File "C:\Users\pavan\anaconda3\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 372, in process
  File "C:\Users\pavan\anaconda3\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\serializers.py", line 352, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "C:\Users\pavan\anaconda3\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\serializers.py", line 142, in dump_stream
    for obj in iterator:
  File "C:\Users\pavan\anaconda3\Lib\site-packages\pyspark\python\lib

In [14]:
train_df, test_df = corpus_df.randomSplit([0.75, 0.25])

In [15]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer, Tokenizer, StopWordsRemover

tokenizer = Tokenizer(inputCol="value", outputCol="words")
stop_words_remover = StopWordsRemover(inputCol="words", outputCol="words_cleaned")
vectorizer = CountVectorizer(inputCol="words_cleaned", outputCol="features")
cleaning_pipeline = Pipeline(stages = [tokenizer,stop_words_remover,vectorizer])
cleaning_pipeline_model = cleaning_pipeline.fit(corpus_df)
cleaned_training_df = cleaning_pipeline_model.transform(train_df)
cleaned_testing_df = cleaning_pipeline_model.transform(test_df)

Py4JJavaError: An error occurred while calling o201.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 1 times, most recent failure: Lost task 0.0 in stage 6.0 (TID 10, localhost, executor driver): java.net.SocketException: Connection reset by peer: socket write error
	at java.net.SocketOutputStream.socketWrite0(Native Method)
	at java.net.SocketOutputStream.socketWrite(Unknown Source)
	at java.net.SocketOutputStream.write(Unknown Source)
	at java.io.BufferedOutputStream.flushBuffer(Unknown Source)
	at java.io.BufferedOutputStream.write(Unknown Source)
	at java.io.DataOutputStream.write(Unknown Source)
	at java.io.FilterOutputStream.write(Unknown Source)
	at org.apache.spark.api.python.PythonRDD$.org$apache$spark$api$python$PythonRDD$$write$1(PythonRDD.scala:212)
	at org.apache.spark.api.python.PythonRDD$$anonfun$writeIteratorToStream$1.apply(PythonRDD.scala:224)
	at org.apache.spark.api.python.PythonRDD$$anonfun$writeIterator

In [ ]:
train_df.show()

In [ ]:
cleaned_training_df.show()

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import Word2Vec, Tokenizer, StopWordsRemover

tokenizer = Tokenizer(inputCol="value", outputCol="words")
stop_words_remover = StopWordsRemover(inputCol="words", outputCol="words_cleaned")
word2vectorizer = Word2Vec(inputCol="words_cleaned", outputCol="features")
cleaning_pipeline = Pipeline(stages = [tokenizer,stop_words_remover,word2vectorizer])
cleaning_pipeline_model = cleaning_pipeline.fit(corpus_df)
cleaned_training_w2v_df = cleaning_pipeline_model.transform(train_df)
cleaned_testing_w2v_df = cleaning_pipeline_model.transform(test_df)

In [ ]:
cleaned_training_df.show()

In [ ]:
from pyspark.ml.classification import NaiveBayes,LogisticRegression,OneVsRest
naive_bayes = NaiveBayes(featuresCol="features", labelCol = "label")

In [ ]:
naive_bayes_model = naive_bayes.fit(cleaned_training_df)
predictions_df = naive_bayes_model.transform(cleaned_testing_df)

In [ ]:
predictions_df.select("features","label","prediction").limit(20).show()

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
eval = MulticlassClassificationEvaluator(labelCol='label',predictionCol='prediction', metricName = 'accuracy')
eval.evaluate(predictions_df)

In [ ]:
cleaned_training_df.select("features").show(4)

In [ ]:
log_reg = LogisticRegression(featuresCol="features", labelCol = "label")
ovr=OneVsRest(classifier=log_reg)
log_reg_model = ovr.fit(cleaned_training_w2v_df)
predictions_w2v_df = log_reg_model.transform(cleaned_testing_w2v_df)

In [ ]:
predictions_w2v_df.select("features","label","prediction").limit(20).show()

In [ ]:
eval.evaluate(predictions_w2v_df)

In [ ]:
corpus_df=corpus_df.toPandas()
#corpus_df.to_csv('tweets_corpus.csv',ignore_index=True)
def add_prefix(colum):
    return "__label__"+str(colum)
corpus_df["value"]=corpus_df["label"].apply(add_prefix)+" "+corpus_df["value"]


In [ ]:
#corpus_df['value'] = corpus_df['value'].apply(lambda x: x.split())

In [ ]:
corpus_df

In [ ]:
corpus_df['value'].to_csv("tweets_corpus.csv",index=False)

In [ ]:
import fasttext

In [ ]:
train_df=train_df.toPandas()
test_df=test_df.toPandas()

In [ ]:
train_df["value"]=train_df["label"].apply(add_prefix)+" "+train_df["value"]
test_df["value"]=test_df["label"].apply(add_prefix)+" "+test_df["value"]

In [ ]:
train_df['value'].to_csv("tweets_train_corpus.csv",index=False)
test_df['value'].to_csv("tweets_test_corpus.csv",index=False)

In [ ]:
model.labels

In [ ]:
model = fasttext.train_supervised('tweets_train_corpus.csv')

In [ ]:
model.labels

In [ ]:
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p)) #Precision
    print("R@{}\t{:.3f}".format(1, r)) #Recall

In [ ]:
print_results(*model.test('tweets_test_corpus.csv'))

In [ ]:
model = fasttext.train_supervised(input="tweets_train_corpus.csv")

In [ ]:
model.test('tweets_test_corpus.csv')

In [ ]:
#The precision is the number of correct labels among the labels predicted by fastText.
#The recall is the number of labels that successfully were predicted, among all the real labels.

In [ ]:
model = fasttext.train_supervised(input="tweets_train_corpus.csv", lr=0.5, epoch=40, wordNgrams=2, bucket=200000, dim=50, loss='ova')

In [ ]:
model.test('tweets_test_corpus.csv')

In [ ]:
model = fasttext.train_unsupervised('tweets_corpus.csv')

In [ ]:
model.words